<a href="https://colab.research.google.com/github/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting/blob/main/notebooks/02_group_stat_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GroupStat Model Overview


This notebook introduces the **GroupStat model**, a structured approach to feature engineering based on **group-level target statistics**. The goal is to enhance predictive performance by incorporating historical patterns observed within business hierarchies — specifically, `Store` and `Dept`.

### Motivation

In retail forecasting tasks, each `Store` and `Department` often follows its own trend. Instead of relying purely on raw input features, we predict meaningful statistical signals — such as **average group sales** — and use them as model inputs. These statistics reflect stable behavior at the group level and serve as strong, informative priors.

### Methodology

The approach involves the following steps:

1. **Group** data by `Store` and `Dept`.
2. **Predict the average of the target variable** (`Weekly_Sales`) for each group — at time `t`.
3. **Assign these averages** as new features:
   - `Store_Prediction`: mean target for the corresponding Store.
   - `Dept_Prediction`: mean target for the corresponding Dept.
4. **Train a final predictive model** (e.g., XGBoost, LightGBM) using the enriched dataset.

### Example Feature Table

| Date       | Store | Dept | Weekly_Sales | Store_Prediction | Dept_Prediction |
|------------|-------|------|--------------|------------------|-----------------|
| 2011-02-11 |   5   |  2   | 12,000        | 14,300           | 13,200          |

These added features provide strong contextual signals:
- `Store_Prediction` captures the general sales level of a store at some time `t`.
- `Dept_Prediction` reflects typical sales volume for a department across stores.

### Final Modeling

We use **gradient boosting models** like **XGBoost** and **LightGBM**, which are capable of modeling complex interactions between raw features and these group-level signals. By incorporating `Dept_Prediction` and `Store_Prediction`, the model learns both fine-grained and coarse patterns — improving both stability and accuracy.

# Notebook Setup

The following setup is provided as a basic example for initializing the notebook environment. It includes necessary imports, optional configuration, and a placeholder for data loading or downloading.

This section is **not part of the core model logic**, and the code here may vary depending on your environment or data access method.

## Setup Environment


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import userdata
token = userdata.get('GITHUB_TOKEN')
user_name = userdata.get('GITHUB_USERNAME')
mail = userdata.get('GITHUB_MAIL')

!git config --global user.name "{user_name}"
!git config --global user.email "{mail}"
!git clone https://{token}@github.com/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting.git

%cd Walmart-Recruiting-Store-Sales-Forecasting

Cloning into 'Walmart-Recruiting-Store-Sales-Forecasting'...
remote: Enumerating objects: 338, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 338 (delta 68), reused 48 (delta 21), pack-reused 209 (from 1)
Receiving objects: 100% (338/338), 6.90 MiB | 15.57 MiB/s, done.
Resolving deltas: 100% (165/165), done.
/content/Walmart-Recruiting-Store-Sales-Forecasting


In [6]:
!pip install -r requirements.txt

In [4]:
from google.colab import userdata
kaggle_json_path = userdata.get('KAGGLE_JSON_PATH')
! ./src/data_loader.sh -f {kaggle_json_path}

Setting up Kaggle credentials...
Ensuring data directory exists at 'data/'...
  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 696MB/s]
Unzipping files...
Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           
Archive:  features.csv.zip
  inflating: features.csv            
Archive:  sampleSubmission.csv.zip
  inflating: sampleSubmission.csv    
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               
Data downloaded and extracted successfully to 'data/'.


In [5]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: zhorzholianimate (MLBeasts) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Load and Split Data

In [7]:
from src import data_loader, processing
import importlib
importlib.reload(processing)

dataframes = data_loader.load_raw_data()
df = processing.run_preprocessing(dataframes, process_test=False)['train']
X_train, y_train, X_valid, y_valid = processing.split_data(df, separate_target=True)

print(f"Shapes of X_train and y_train: {X_train.shape}, {y_train.shape}")
print(f"Shapes of X_valid and y_valid: {X_valid.shape}, {y_valid.shape}")

Data loading complete.
Shapes of X_train and y_train: (279085, 15), (279085,)
Shapes of X_valid and y_valid: (142485, 15), (142485,)


In [8]:
X_train

,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1,1,2010-02-05,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
1,1,2,2010-02-05,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
2,1,3,2010-02-05,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
3,1,4,2010-02-05,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
4,1,5,2010-02-05,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279080,45,93,2011-11-25,True,48.71,3.492,140.87,384.82,26961.99,28.59,1110.12,188.350400,8.523,B,118221
279081,45,94,2011-11-25,True,48.71,3.492,140.87,384.82,26961.99,28.59,1110.12,188.350400,8.523,B,118221
279082,45,95,2011-11-25,True,48.71,3.492,140.87,384.82,26961.99,28.59,1110.12,188.350400,8.523,B,118221
279083,45,97,2011-11-25,True,48.71,3.492,140.87,384.82,26961.99,28.59,1110.12,188.350400,8.523,B,118221


In [9]:
X_valid

,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
279085,1,1,2011-12-02,False,48.91,3.172,5629.51,68.00,1398.11,2084.64,20475.32,218.714733,7.866,A,151315
279086,1,2,2011-12-02,False,48.91,3.172,5629.51,68.00,1398.11,2084.64,20475.32,218.714733,7.866,A,151315
279087,1,3,2011-12-02,False,48.91,3.172,5629.51,68.00,1398.11,2084.64,20475.32,218.714733,7.866,A,151315
279088,1,4,2011-12-02,False,48.91,3.172,5629.51,68.00,1398.11,2084.64,20475.32,218.714733,7.866,A,151315
279089,1,5,2011-12-02,False,48.91,3.172,5629.51,68.00,1398.11,2084.64,20475.32,218.714733,7.866,A,151315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421565,45,93,2012-10-26,False,58.85,3.882,4018.91,58.08,100.00,211.94,858.33,192.308899,8.667,B,118221
421566,45,94,2012-10-26,False,58.85,3.882,4018.91,58.08,100.00,211.94,858.33,192.308899,8.667,B,118221
421567,45,95,2012-10-26,False,58.85,3.882,4018.91,58.08,100.00,211.94,858.33,192.308899,8.667,B,118221
421568,45,97,2012-10-26,False,58.85,3.882,4018.91,58.08,100.00,211.94,858.33,192.308899,8.667,B,118221


# Training & Inference Logic

## Predicting Store-Level Averages with XGBoost


In this section, we build an XGBoost regression model to predict the **average target value per Store**, which we refer to as `Store_Prediction`.

The model uses a comprehensive set of store-specific features, including aggregated and static attributes that characterize each store.

Additionally, we incorporate two key enhancements:

1. **Store_mean**: the mean sales per store calculated over the training period. This feature reflects the overall scale of each store’s business and provides valuable context to explain its average performance.
2. **Time-related features**: such as month, week of the year, and day-of-week indicators, all derived from the date. These features enable the model to capture seasonal and temporal patterns in store behavior.

> **Note:** Time-related features will be included consistently across all models in this notebook, so this point will not be repeated in later sections.

We apply hyperparameter tuning using cross-validation to optimize predictive accuracy and generalization. The resulting XGBoost model serves as the **Store-level predictor** within the GroupStat framework.



Start by grouping the data at the store level and performing aggregation.

In [56]:
from feature_engineering import grouper as grp
import importlib
importlib.reload(grp)

X_train_store, y_train_store = grp.group_and_aggregate(X_train, y_train, groupby_cols=['Store', 'Date', 'IsHoliday'], y_aggfunc='mean')
X_valid_store, y_valid_store = grp.group_and_aggregate(X_valid, y_valid, groupby_cols=['Store', 'Date', 'IsHoliday'], y_aggfunc='mean')

X_train_store

,Store,Date,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1,2010-02-05,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
1,1,2010-02-12,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,A,151315
2,1,2010-02-19,False,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,A,151315
3,1,2010-02-26,False,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,A,151315
4,1,2010-03-05,False,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,A,151315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4270,45,2011-10-28,False,51.78,3.569,NaN,NaN,NaN,NaN,NaN,187.877491,8.523,B,118221
4271,45,2011-11-04,False,43.92,3.551,NaN,NaN,NaN,NaN,NaN,187.970363,8.523,B,118221
4272,45,2011-11-11,False,47.65,3.530,23052.25,5449.62,189.24,3355.69,3864.60,188.063234,8.523,B,118221
4273,45,2011-11-18,False,51.34,3.530,4240.34,132.96,111.71,270.14,7073.00,188.198365,8.523,B,118221


Apply data transformations such as adding time-based features, converting object columns to categorical, and setting store as a categorical variable, dropping markdowns.

In [57]:
from feature_engineering import time_features, feature_transformers

import importlib
importlib.reload(time_features)
importlib.reload(feature_transformers)

params = {
    'add_week_num' : True,
    'add_holiday_flags' : True,
    'add_holiday_proximity': True,
    'add_holiday_windows': True,
    'add_fourier_features': True,
    'add_month_and_year': True,
    'replace_time_index': True,
}

feature_adder = time_features.FeatureAdder(**params)
X_train_store = feature_adder.fit_transform(X_train_store)
X_valid_store = feature_adder.transform(X_valid_store)

object_transformer = feature_transformers.ObjectToCategory()
X_train_store = object_transformer.fit_transform(X_train_store)
X_valid_store = object_transformer.transform(X_valid_store)

adder = feature_transformers.GroupStatFeatureAdder(groupby_cols='Store', aggfunc='mean')
X_train_store = adder.fit_transform(X_train_store, y_train_store)
X_valid_store = adder.transform(X_valid_store)

make_categorical_transformer = feature_transformers.MakeCategorical(['Store'])
X_train_store = make_categorical_transformer.fit_transform(X_train_store)
X_valid_store = make_categorical_transformer.transform(X_valid_store)

columns_to_drop=['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
change_columns_transformer = feature_transformers.ChangeColumns(columns_to_drop=columns_to_drop)
X_train_store = change_columns_transformer.fit_transform(X_train_store)
X_valid_store = change_columns_transformer.transform(X_valid_store)

X_valid_store

,Store,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment,Type,Size,Month,Year,...,Days_until_next_Thanksgiving,Days_since_last_Thanksgiving,Days_until_next_Christmas,Days_since_last_Christmas,Days_until_next_SuperBowl,Days_since_last_SuperBowl,Days_until_next_LaborDay,Days_since_last_LaborDay,Date,Store_mean
0,1,False,48.91,3.172,218.714733,7.866,A,151315,12,2011,...,357,7,28,336,70,294,280,84,95,21335.179178
1,1,False,43.93,3.158,218.961846,7.866,A,151315,12,2011,...,350,14,21,343,63,301,273,91,96,21335.179178
2,1,False,51.63,3.159,219.179453,7.866,A,151315,12,2011,...,343,21,14,350,56,308,266,98,97,21335.179178
3,1,False,47.96,3.112,219.357722,7.866,A,151315,12,2011,...,336,28,7,357,49,315,259,105,98,21335.179178
4,1,True,44.55,3.129,219.535990,7.866,A,151315,12,2011,...,329,35,0,364,42,322,252,112,99,21335.179178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,45,False,64.88,3.997,192.013558,8.684,B,118221,9,2012,...,56,308,91,273,133,231,343,21,138,11639.767411
2156,45,False,64.89,3.985,192.170412,8.667,B,118221,10,2012,...,49,315,84,280,126,238,336,28,139,11639.767411
2157,45,False,54.47,4.000,192.327265,8.667,B,118221,10,2012,...,42,322,77,287,119,245,329,35,140,11639.767411
2158,45,False,56.47,3.969,192.330854,8.667,B,118221,10,2012,...,35,329,70,294,112,252,322,42,141,11639.767411


We perform two rounds of cross-validation to tune XGBoost:

1. Tune main parameters: `n_estimators`, `max_depth`, `learning_rate`.  
2. Fix those and tune regularization parameters: `subsample`, `colsample_bytree`, `min_child_weight`.

In [ ]:
from xgboost import XGBRegressor
from src.utils import wmae
from src.cross_validation import manual_model_search

model = XGBRegressor(
    objective='reg:squarederror',
    enable_categorical=True,
    random_state=42
)

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7, 10],
}


metric_kwargs = {
    'is_holiday': X_valid_store['IsHoliday']
}

best_model, best_params, best_score = manual_model_search(
    model=model,
    param_grid=param_grid,
    X_train=X_train_store,
    y_train=y_train_store,
    X_valid=X_valid_store,
    y_valid=y_valid_store,
    metric_func=wmae,
    metric_kwargs=metric_kwargs
)

print("\nBest Params:", best_params)
print("Best Validation Score:", best_score)

Params: {'n_estimators': 100, 'learning_rate': 0.01, 'max_depth': 3} -> Score: 2577.2952
Params: {'n_estimators': 100, 'learning_rate': 0.01, 'max_depth': 5} -> Score: 2548.0074
Params: {'n_estimators': 100, 'learning_rate': 0.01, 'max_depth': 7} -> Score: 2528.4732
Params: {'n_estimators': 100, 'learning_rate': 0.01, 'max_depth': 10} -> Score: 2508.5963
Params: {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 3} -> Score: 1085.9913
Params: {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 5} -> Score: 942.5534
Params: {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 7} -> Score: 898.6599
Params: {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 10} -> Score: 905.4130
Params: {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 3} -> Score: 918.9623
Params: {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 5} -> Score: 842.9912
Params: {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 7} -> Score: 846.4202
Params: {'n_estimators': 100

In [12]:
from xgboost import XGBRegressor
from src.utils import wmae
from src.cross_validation import manual_model_search

model = XGBRegressor(
    objective='reg:squarederror',
    enable_categorical=True,
    random_state=42,
    n_estimators=200,
    learning_rate=0.1,
    max_depth=7,
)

param_grid = {
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'min_child_weight': [1, 5, 10]
}

metric_kwargs = {
    'is_holiday': X_valid_store['IsHoliday']
}

best_model, best_params, best_score = manual_model_search(
    model=model,
    param_grid=param_grid,
    X_train=X_train_store,
    y_train=y_train_store,
    X_valid=X_valid_store,
    y_valid=y_valid_store,
    metric_func=wmae,
    metric_kwargs=metric_kwargs
)

print("\nBest Params:", best_params)
print("Best Validation Score:", best_score)

Params: {'subsample': 0.6, 'colsample_bytree': 0.5, 'min_child_weight': 1} -> Score: 939.2996
Params: {'subsample': 0.6, 'colsample_bytree': 0.5, 'min_child_weight': 5} -> Score: 848.4098
Params: {'subsample': 0.6, 'colsample_bytree': 0.5, 'min_child_weight': 10} -> Score: 866.5353
Params: {'subsample': 0.6, 'colsample_bytree': 0.7, 'min_child_weight': 1} -> Score: 841.2455
Params: {'subsample': 0.6, 'colsample_bytree': 0.7, 'min_child_weight': 5} -> Score: 808.4864
Params: {'subsample': 0.6, 'colsample_bytree': 0.7, 'min_child_weight': 10} -> Score: 837.7769
Params: {'subsample': 0.6, 'colsample_bytree': 1.0, 'min_child_weight': 1} -> Score: 848.4305
Params: {'subsample': 0.6, 'colsample_bytree': 1.0, 'min_child_weight': 5} -> Score: 800.0636
Params: {'subsample': 0.6, 'colsample_bytree': 1.0, 'min_child_weight': 10} -> Score: 820.8207
Params: {'subsample': 0.8, 'colsample_bytree': 0.5, 'min_child_weight': 1} -> Score: 939.8671
Params: {'subsample': 0.8, 'colsample_bytree': 0.5, 'min_

Train the final XGBoost model on the entire training dataset using the best hyperparameters.  
Then generate the `Store_Prediction` feature by applying this model, which will be used as an additional input in the main dataset for downstream modeling.

In [58]:
from re import sub
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from src.utils import wmae as compute_wmae

store_model = XGBRegressor(
    objective='reg:squarederror',
    enable_categorical=True,
    random_state=42,
    n_estimators=200,
    learning_rate=0.1,
    max_depth=7,
    subsample=0.6,
    colsample_bytree=1.0,
    min_child_weight=5
)

store_model.fit(X_train_store, y_train_store)
store_train_preds = store_model.predict(X_train_store)
store_valid_preds = store_model.predict(X_valid_store)

valid_wmae = compute_wmae(y_valid_store, store_valid_preds, is_holiday=X_valid_store['IsHoliday'])
print(f"Validation WMAE: {valid_wmae:.2f}")
valid_mae = mean_absolute_error(y_valid_store, store_valid_preds)
print(f"Validation MAE: {valid_mae:.2f}")

train_wmae = compute_wmae(y_train_store, store_train_preds, is_holiday=X_train_store['IsHoliday'])
print(f"Train WMAE: {train_wmae:.2f}")
train_mae = mean_absolute_error(y_train_store, store_train_preds)
print(f"Train MAE: {train_mae:.2f}")


Validation WMAE: 800.06
Validation MAE: 786.72
Train WMAE: 253.43
Train MAE: 254.05


In [59]:
valid_store_results = X_valid_store[['Store', 'Date']].copy()
valid_store_results['Store_Prediction'] = store_valid_preds

train_store_results = X_train_store[['Store', 'Date']].copy()
train_store_results['Store_Prediction'] = store_train_preds

valid_store_results

,Store,Date,Store_Prediction
0,1,95,23446.662109
1,1,96,24663.173828
2,1,97,26697.953125
3,1,98,34912.000000
4,1,99,19632.361328
...,...,...,...
2155,45,138,10711.922852
2156,45,139,11598.338867
2157,45,140,11298.853516
2158,45,141,11547.879883


Log the result on the WandB.

In [60]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('feature_adder', time_features.FeatureAdder(**params)),
    ('object_to_category', feature_transformers.ObjectToCategory()),
    ('group_stat_features', feature_transformers.GroupStatFeatureAdder(groupby_cols='Store', aggfunc='mean')),
    ('make_categorical', feature_transformers.MakeCategorical(['Store'])),
    ('drop_columns', feature_transformers.ChangeColumns(columns_to_drop=[
        'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'
    ])),
    ('model', store_model)
])

from src.utils import log_to_wandb
from configs.basic_config import store_avg_config
from configs.boosting_models_config import store_avg_xgboost_config



log_to_wandb(
    model = pipeline,
    train_score = train_wmae,
    val_score = valid_wmae,
    run_name = "store_avg_xgboost01",
    artifact_type = "model",
    artifact_name="store_avg_boosting",
    artifact_description = "used for predicting stores avg only",
    config= store_avg_config | store_avg_xgboost_config)



train_wmae,▁
val_wmae,▁
train_wmae,253.43264
val_wmae,800.06355


## Predicting Department-Level Averages with XGBoost




Here, we build an XGBoost model to predict the **average target value per Department** (`Dept_Prediction`).

Unlike the store-level model, most available features are store-specific and thus not useful for predicting department-level behavior. Therefore, this model relies primarily on department-level aggregated features and additionam time features.

The model is tuned via cross-validation to ensure good generalization and forms the department-level component of the GroupStat framework.

Start by grouping the data at the department level and performing aggregation.

In [61]:
from feature_engineering import grouper as grp
import importlib
importlib.reload(grp)

X_train_dept, y_train_dept = grp.group_and_aggregate(X_train, y_train, groupby_cols=['Date', 'IsHoliday', 'Dept'], y_aggfunc='mean')
X_valid_dept, y_valid_dept = grp.group_and_aggregate(X_valid, y_valid, groupby_cols=['Date', 'IsHoliday', 'Dept'], y_aggfunc='mean')

adder = feature_transformers.GroupStatFeatureAdder(groupby_cols='Dept', aggfunc='mean')
X_train_dept = adder.fit_transform(X_train_dept, y_train_dept)
X_valid_dept = adder.transform(X_valid_dept)

X_train_dept

,Date,IsHoliday,Dept,Dept_mean
0,2010-02-05,False,1,18960.208250
1,2010-02-05,False,2,43193.216126
2,2010-02-05,False,3,11592.413923
3,2010-02-05,False,4,25743.993319
4,2010-02-05,False,5,22271.562580
...,...,...,...,...
7335,2011-11-25,True,95,69428.162894
7336,2011-11-25,True,96,15280.287656
7337,2011-11-25,True,97,14176.271374
7338,2011-11-25,True,98,7004.618635


Apply data transformations such as adding time-based features, converting object columns to categorical, and setting departments as a categorical variable..

In [62]:
from feature_engineering import time_features, feature_transformers

import importlib
importlib.reload(time_features)
importlib.reload(feature_transformers)

params = {
    'add_week_num' : True,
    'add_holiday_flags' : True,
    'add_holiday_proximity': True,
    'add_holiday_windows': True,
    'add_fourier_features': True,
    'add_month_and_year': True,
    'replace_time_index': True,
}

feature_adder = time_features.FeatureAdder(**params)
X_train_dept = feature_adder.fit_transform(X_train_dept)
X_valid_dept = feature_adder.transform(X_valid_dept)

object_transformer = feature_transformers.ObjectToCategory()
X_train_dept = object_transformer.fit_transform(X_train_dept)
X_valid_dept = object_transformer.transform(X_valid_dept)

make_categorical_transformer = feature_transformers.MakeCategorical(['Dept'])
X_train_dept = make_categorical_transformer.fit_transform(X_train_dept)
X_valid_dept = make_categorical_transformer.transform(X_valid_dept)


X_train_dept

,IsHoliday,Dept,Dept_mean,Month,Year,WeekOfYear,Is_Thanksgiving,Is_Christmas,Is_SuperBowl,Is_LaborDay,...,month_cos,Days_until_next_Thanksgiving,Days_since_last_Thanksgiving,Days_until_next_Christmas,Days_since_last_Christmas,Days_until_next_SuperBowl,Days_since_last_SuperBowl,Days_until_next_LaborDay,Days_since_last_LaborDay,Date
0,False,1,18960.208250,2,2010,5,0,0,0,0,...,0.500000,294,999.0,329,999.0,7,999.0,217,999.0,0
1,False,2,43193.216126,2,2010,5,0,0,0,0,...,0.500000,294,999.0,329,999.0,7,999.0,217,999.0,0
2,False,3,11592.413923,2,2010,5,0,0,0,0,...,0.500000,294,999.0,329,999.0,7,999.0,217,999.0,0
3,False,4,25743.993319,2,2010,5,0,0,0,0,...,0.500000,294,999.0,329,999.0,7,999.0,217,999.0,0
4,False,5,22271.562580,2,2010,5,0,0,0,0,...,0.500000,294,999.0,329,999.0,7,999.0,217,999.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7335,True,95,69428.162894,11,2011,47,1,0,0,0,...,0.866025,0,364.0,35,329.0,77,287.0,287,77.0,94
7336,True,96,15280.287656,11,2011,47,1,0,0,0,...,0.866025,0,364.0,35,329.0,77,287.0,287,77.0,94
7337,True,97,14176.271374,11,2011,47,1,0,0,0,...,0.866025,0,364.0,35,329.0,77,287.0,287,77.0,94
7338,True,98,7004.618635,11,2011,47,1,0,0,0,...,0.866025,0,364.0,35,329.0,77,287.0,287,77.0,94


As above, we perform two rounds of cross-validation to tune XGBoost.

In [ ]:
from xgboost import XGBRegressor
from src.utils import wmae
from src.cross_validation import manual_model_search

model = XGBRegressor(
    objective='reg:squarederror',
    enable_categorical=True,
    random_state=42
)

param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
}


metric_kwargs = {
    'is_holiday': X_valid_dept['IsHoliday']
}

best_model, best_params, best_score = manual_model_search(
    model=model,
    param_grid=param_grid,
    X_train=X_train_dept,
    y_train=y_train_dept,
    X_valid=X_valid_dept,
    y_valid=y_valid_dept,
    metric_func=wmae,
    metric_kwargs=metric_kwargs
)

print("\nBest Params:", best_params)
print("Best Validation Score:", best_score)

Params: {'n_estimators': 50, 'learning_rate': 0.01, 'max_depth': 3} -> Score: 8121.7384
Params: {'n_estimators': 50, 'learning_rate': 0.01, 'max_depth': 5} -> Score: 8058.3132
Params: {'n_estimators': 50, 'learning_rate': 0.01, 'max_depth': 7} -> Score: 7969.8613
Params: {'n_estimators': 50, 'learning_rate': 0.05, 'max_depth': 3} -> Score: 2688.5536
Params: {'n_estimators': 50, 'learning_rate': 0.05, 'max_depth': 5} -> Score: 2257.7474
Params: {'n_estimators': 50, 'learning_rate': 0.05, 'max_depth': 7} -> Score: 2017.7425
Params: {'n_estimators': 50, 'learning_rate': 0.1, 'max_depth': 3} -> Score: 1873.3665
Params: {'n_estimators': 50, 'learning_rate': 0.1, 'max_depth': 5} -> Score: 1502.4315
Params: {'n_estimators': 50, 'learning_rate': 0.1, 'max_depth': 7} -> Score: 1297.3211
Params: {'n_estimators': 100, 'learning_rate': 0.01, 'max_depth': 3} -> Score: 5476.6441
Params: {'n_estimators': 100, 'learning_rate': 0.01, 'max_depth': 5} -> Score: 5379.8402
Params: {'n_estimators': 100, 'le

In [ ]:
from xgboost import XGBRegressor
from src.utils import wmae
from src.cross_validation import manual_model_search

model = XGBRegressor(
    objective='reg:squarederror',
    enable_categorical=True,
    random_state=42,
    n_estimators=200,
    learning_rate=0.1,
    max_depth=7,
)

param_grid = {
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'min_child_weight': [1, 5, 10],
    'n_estimators': [200, 300],
}

metric_kwargs = {
    'is_holiday': X_valid_dept['IsHoliday']
}

best_model, best_params, best_score = manual_model_search(
    model=model,
    param_grid=param_grid,
    X_train=X_train_dept,
    y_train=y_train_dept,
    X_valid=X_valid_dept,
    y_valid=y_valid_dept,
    metric_func=wmae,
    metric_kwargs=metric_kwargs
)

print("\nBest Params:", best_params)
print("Best Validation Score:", best_score)

Params: {'subsample': 0.6, 'colsample_bytree': 0.5, 'min_child_weight': 1, 'n_estimators': 200} -> Score: 1401.3283
Params: {'subsample': 0.6, 'colsample_bytree': 0.5, 'min_child_weight': 1, 'n_estimators': 300} -> Score: 1386.3900
Params: {'subsample': 0.6, 'colsample_bytree': 0.5, 'min_child_weight': 5, 'n_estimators': 200} -> Score: 1406.2531
Params: {'subsample': 0.6, 'colsample_bytree': 0.5, 'min_child_weight': 5, 'n_estimators': 300} -> Score: 1372.4439
Params: {'subsample': 0.6, 'colsample_bytree': 0.5, 'min_child_weight': 10, 'n_estimators': 200} -> Score: 1618.1366
Params: {'subsample': 0.6, 'colsample_bytree': 0.5, 'min_child_weight': 10, 'n_estimators': 300} -> Score: 1553.6421
Params: {'subsample': 0.6, 'colsample_bytree': 0.7, 'min_child_weight': 1, 'n_estimators': 200} -> Score: 1497.9736
Params: {'subsample': 0.6, 'colsample_bytree': 0.7, 'min_child_weight': 1, 'n_estimators': 300} -> Score: 1488.0840
Params: {'subsample': 0.6, 'colsample_bytree': 0.7, 'min_child_weight'

Train the final XGBoost model on the entire training dataset and get `Prediction_Dept`.

In [63]:
from re import sub
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from src.utils import wmae as compute_wmae

dept_model = XGBRegressor(
    objective='reg:squarederror',
    enable_categorical=True,
    random_state=42,
    n_estimators=300,
    learning_rate=0.1,
    max_depth=7,
    subsample=1.0,
    colsample_bytree=0.5,
    min_child_weight=1
)

dept_model.fit(X_train_dept, y_train_dept)
dept_train_preds = dept_model.predict(X_train_dept)
dept_valid_preds = dept_model.predict(X_valid_dept)

valid_wmae = compute_wmae(y_valid_dept, dept_valid_preds, is_holiday=X_valid_dept['IsHoliday'])
print(f"Validation WMAE: {valid_wmae:.2f}")
valid_mae = mean_absolute_error(y_valid_dept, dept_valid_preds)
print(f"Validation MAE: {valid_mae:.2f}")

train_wmae = compute_wmae(y_train_dept, dept_train_preds, is_holiday=X_train_dept['IsHoliday'])
print(f"Train WMAE: {train_wmae:.2f}")
train_mae = mean_absolute_error(y_train_dept, dept_train_preds)
print(f"Train MAE: {train_mae:.2f}")

Validation WMAE: 1060.54
Validation MAE: 1017.28
Train WMAE: 263.01
Train MAE: 276.83


In [64]:
valid_dept_results = X_valid_dept[['Dept', 'Date']].copy()
valid_dept_results['Dept_Prediction'] = dept_valid_preds

train_dept_results = X_train_dept[['Dept', 'Date']].copy()
train_dept_results['Dept_Prediction'] = dept_train_preds

valid_dept_results

,Dept,Date,Dept_Prediction
0,1,95,21831.798828
1,2,95,41211.687500
2,3,95,9007.485352
3,4,95,25928.740234
4,5,95,37909.156250
...,...,...,...
3745,95,142,65300.281250
3746,96,142,15349.022461
3747,97,142,13490.367188
3748,98,142,6990.856934


Log the result on the Wandb.

In [65]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('feature_adder', time_features.FeatureAdder(**params)),
    ('object_to_category', feature_transformers.ObjectToCategory()),
    ('make_categorical', feature_transformers.MakeCategorical(['Dept'])),
    ('model', dept_model)
])


from src.utils import log_to_wandb
from configs.basic_config import dept_avg_config
from configs.boosting_models_config import dept_avg_xgboost_config

log_to_wandb(
    model = pipeline,
    train_score = train_wmae,
    val_score = valid_wmae,
    run_name = "dept_avg_xgboost01",
    artifact_type = "model",
    artifact_name="dept_avg_boosting",
    artifact_description = "used for predicting dept avg only",
    config= dept_avg_config | dept_avg_xgboost_config)

train_wmae,▁
val_wmae,▁
train_wmae,263.01149
val_wmae,1060.53556


## Final Model Using Group-Level Predictions

In this stage, we incorporate the previously trained **store-level** and **department-level** predictions as additional features in the dataset. These aggregated statistics serve as powerful priors, helping the model capture group-specific patterns.

Given the increased dataset size after feature augmentation, we choose to use **LightGBM**, which is optimized for large-scale data and offers efficient training speed and memory usage.

Apply data transformations such as adding time-based features, converting object columns to categorical, and setting departments and stores as a categorical variable.

In [66]:
params = {
    'add_week_num' : True,
    'add_holiday_flags' : True,
    'add_holiday_proximity': True,
    'add_holiday_windows': True,
    'add_fourier_features': True,
    'add_month_and_year': True,
    'replace_time_index': True,
}

feature_adder = time_features.FeatureAdder(**params)
X_train_t = feature_adder.fit_transform(X_train)
X_valid_t = feature_adder.transform(X_valid)

object_transformer = feature_transformers.ObjectToCategory()
X_train_t = object_transformer.fit_transform(X_train_t)
X_valid_t = object_transformer.transform(X_valid_t)

make_categorical_transformer = feature_transformers.MakeCategorical(['Dept', 'Store'])
X_train_t = make_categorical_transformer.fit_transform(X_train_t)
X_valid_t = make_categorical_transformer.transform(X_valid_t)


X_train_t

,Store,Dept,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,...,month_cos,Days_until_next_Thanksgiving,Days_since_last_Thanksgiving,Days_until_next_Christmas,Days_since_last_Christmas,Days_until_next_SuperBowl,Days_since_last_SuperBowl,Days_until_next_LaborDay,Days_since_last_LaborDay,Date
0,1,1,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,...,0.500000,294,999.0,329,999.0,7,999.0,217,999.0,0
1,1,2,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,...,0.500000,294,999.0,329,999.0,7,999.0,217,999.0,0
2,1,3,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,...,0.500000,294,999.0,329,999.0,7,999.0,217,999.0,0
3,1,4,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,...,0.500000,294,999.0,329,999.0,7,999.0,217,999.0,0
4,1,5,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,...,0.500000,294,999.0,329,999.0,7,999.0,217,999.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279080,45,93,True,48.71,3.492,140.87,384.82,26961.99,28.59,1110.12,...,0.866025,0,364.0,35,329.0,77,287.0,287,77.0,94
279081,45,94,True,48.71,3.492,140.87,384.82,26961.99,28.59,1110.12,...,0.866025,0,364.0,35,329.0,77,287.0,287,77.0,94
279082,45,95,True,48.71,3.492,140.87,384.82,26961.99,28.59,1110.12,...,0.866025,0,364.0,35,329.0,77,287.0,287,77.0,94
279083,45,97,True,48.71,3.492,140.87,384.82,26961.99,28.59,1110.12,...,0.866025,0,364.0,35,329.0,77,287.0,287,77.0,94


In [67]:
X_train_T = X_train_t.merge(train_dept_results, on=['Date', 'Dept'], how='left')
X_train_T = X_train_T.merge(train_store_results, on=['Date', 'Store'], how='left')
X_train_T

,Store,Dept,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,...,Days_since_last_Thanksgiving,Days_until_next_Christmas,Days_since_last_Christmas,Days_until_next_SuperBowl,Days_since_last_SuperBowl,Days_until_next_LaborDay,Days_since_last_LaborDay,Date,Dept_Prediction,Store_Prediction
0,1,1,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,...,999.0,329,999.0,7,999.0,217,999.0,0,19838.316406,22419.216797
1,1,2,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,...,999.0,329,999.0,7,999.0,217,999.0,0,44313.085938,22419.216797
2,1,3,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,...,999.0,329,999.0,7,999.0,217,999.0,0,11291.686523,22419.216797
3,1,4,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,...,999.0,329,999.0,7,999.0,217,999.0,0,26536.197266,22419.216797
4,1,5,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,...,999.0,329,999.0,7,999.0,217,999.0,0,25001.996094,22419.216797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279080,45,93,True,48.71,3.492,140.87,384.82,26961.99,28.59,1110.12,...,364.0,35,329.0,77,287.0,287,77.0,94,32321.640625,17340.208984
279081,45,94,True,48.71,3.492,140.87,384.82,26961.99,28.59,1110.12,...,364.0,35,329.0,77,287.0,287,77.0,94,32115.697266,17340.208984
279082,45,95,True,48.71,3.492,140.87,384.82,26961.99,28.59,1110.12,...,364.0,35,329.0,77,287.0,287,77.0,94,64257.125000,17340.208984
279083,45,97,True,48.71,3.492,140.87,384.82,26961.99,28.59,1110.12,...,364.0,35,329.0,77,287.0,287,77.0,94,12546.810547,17340.208984


In [68]:
X_valid_T = X_valid_t.merge(valid_dept_results, on=['Date', 'Dept'], how='left')
X_valid_T = X_valid_T.merge(valid_store_results, on=['Date', 'Store'], how='left')
X_valid_T

,Store,Dept,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,...,Days_since_last_Thanksgiving,Days_until_next_Christmas,Days_since_last_Christmas,Days_until_next_SuperBowl,Days_since_last_SuperBowl,Days_until_next_LaborDay,Days_since_last_LaborDay,Date,Dept_Prediction,Store_Prediction
0,1,1,False,48.91,3.172,5629.51,68.00,1398.11,2084.64,20475.32,...,7,28,336,70,294,280,84,95,21831.798828,23446.662109
1,1,2,False,48.91,3.172,5629.51,68.00,1398.11,2084.64,20475.32,...,7,28,336,70,294,280,84,95,41211.687500,23446.662109
2,1,3,False,48.91,3.172,5629.51,68.00,1398.11,2084.64,20475.32,...,7,28,336,70,294,280,84,95,9007.485352,23446.662109
3,1,4,False,48.91,3.172,5629.51,68.00,1398.11,2084.64,20475.32,...,7,28,336,70,294,280,84,95,25928.740234,23446.662109
4,1,5,False,48.91,3.172,5629.51,68.00,1398.11,2084.64,20475.32,...,7,28,336,70,294,280,84,95,37909.156250,23446.662109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142480,45,93,False,58.85,3.882,4018.91,58.08,100.00,211.94,858.33,...,336,63,301,105,259,315,49,142,25323.859375,11497.519531
142481,45,94,False,58.85,3.882,4018.91,58.08,100.00,211.94,858.33,...,336,63,301,105,259,315,49,142,28883.205078,11497.519531
142482,45,95,False,58.85,3.882,4018.91,58.08,100.00,211.94,858.33,...,336,63,301,105,259,315,49,142,65300.281250,11497.519531
142483,45,97,False,58.85,3.882,4018.91,58.08,100.00,211.94,858.33,...,336,63,301,105,259,315,49,142,13490.367188,11497.519531


Cross-validate the LightGBM model to find the best-performing set of hyperparameters. Also, note that higher values of `n_estimators` tend to yield better results in this setting, likely due to the large dataset size and complexity.

In [ ]:
import lightgbm as lgb

model = lgb.LGBMRegressor(
    objective='regression',
    random_state=42,
    verbose=-1
)

param_grid = {
    'n_estimators': [500, 700, 1000],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [7, 10, 15],
}


metric_kwargs = {
    'is_holiday': X_valid_T['IsHoliday']
}

best_model, best_params, best_score = manual_model_search(
    model=model,
    param_grid=param_grid,
    X_train=X_train_T,
    y_train=y_train,
    X_valid=X_valid_T,
    y_valid=y_valid,
    metric_func=wmae,
    metric_kwargs=metric_kwargs
)

print("\nBest Params:", best_params)
print("Best Validation Score:", best_score)

Params: {'n_estimators': 500, 'learning_rate': 0.05, 'max_depth': 7} -> Score: 2229.2899
Params: {'n_estimators': 500, 'learning_rate': 0.05, 'max_depth': 10} -> Score: 2236.3063
Params: {'n_estimators': 500, 'learning_rate': 0.05, 'max_depth': 15} -> Score: 2239.9927
Params: {'n_estimators': 500, 'learning_rate': 0.1, 'max_depth': 7} -> Score: 2010.4447
Params: {'n_estimators': 500, 'learning_rate': 0.1, 'max_depth': 10} -> Score: 2005.1945
Params: {'n_estimators': 500, 'learning_rate': 0.1, 'max_depth': 15} -> Score: 2013.2947
Params: {'n_estimators': 500, 'learning_rate': 0.2, 'max_depth': 7} -> Score: 1988.8651
Params: {'n_estimators': 500, 'learning_rate': 0.2, 'max_depth': 10} -> Score: 1984.3639
Params: {'n_estimators': 500, 'learning_rate': 0.2, 'max_depth': 15} -> Score: 1982.7143
Params: {'n_estimators': 700, 'learning_rate': 0.05, 'max_depth': 7} -> Score: 2098.9364
Params: {'n_estimators': 700, 'learning_rate': 0.05, 'max_depth': 10} -> Score: 2110.5981
Params: {'n_estimato

In [69]:
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_absolute_error
from src.utils import wmae as compute_wmae

model = lgb.LGBMRegressor(
    objective='regression',
    random_state=42,
    verbose=-1,
    n_estimators=1000,
    learning_rate=0.1,
    max_depth=10
)

model.fit(X_train_T, y_train)

train_preds = model.predict(X_train_T)
valid_preds = model.predict(X_valid_T)

train_wmae = compute_wmae(y_train, train_preds, is_holiday=X_train_T['IsHoliday'])
valid_wmae = compute_wmae(y_valid, valid_preds, is_holiday=X_valid_T['IsHoliday'])
train_mae = mean_absolute_error(y_train, train_preds)
valid_mae = mean_absolute_error(y_valid, valid_preds)

print(f"Train WMAE: {train_wmae:.2f}, MAE: {train_mae:.2f}")
print(f"Valid WMAE: {valid_wmae:.2f}, MAE: {valid_mae:.2f}")


Train WMAE: 1156.92, MAE: 1127.47
Valid WMAE: 1953.42, MAE: 1905.55


Log this final model on the Wandb.

In [70]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('feature_adder', time_features.FeatureAdder(**params)),
    ('object_to_category', feature_transformers.ObjectToCategory()),
    ('make_categorical', feature_transformers.MakeCategorical(['Dept', 'Store'])),
    ("model", model)
])

from configs import boosting_models_config, basic_config
import importlib
importlib.reload(boosting_models_config)
importlib.reload(basic_config)

from src.utils import log_to_wandb
from configs.basic_config import config
from configs.boosting_models_config import lgbm_config

cur_configs = {
    'additional_features' : ['Dept_Prediction', 'Store_Prediction']
}

log_to_wandb(
    model = pipeline,
    train_score = train_wmae,
    val_score = valid_wmae,
    run_name = "lightgbm_for_groupstat01",
    artifact_name="pred_features_boosting",
    artifact_type = "model",
    artifact_description = "Predicts Overall Information using Other models store/dept predictions",
    config= config | lgbm_config | cur_configs)

train_wmae,▁
val_wmae,▁
train_wmae,1156.92342
val_wmae,1953.42277


# Implementation Overview



Based on the analysis and modeling described above, I implemented the **GroupStatModel** model as a general-purpose grouping framework located in `models/group_stat`. This framework provides flexible functionality for grouping stores and departments (but not other hierarchical categories) and computing aggregated statistics used as predictive features.

Using this framework, the specific model for the Walmart sales forecasting task was developed under `models/walmart_group_sales` with the class name `WalmartGroupSalesModel`. This model leverages the GroupStat architecture tailored to the Walmart dataset and forecasting requirements.

Here is training example:

In [55]:
from models import walmart_group_sales
import importlib
importlib.reload(walmart_group_sales)

mdl = walmart_group_sales.WalmartGroupSalesModel()

mdl.fit(X_train, y_train)

pred = mdl.predict(X_train)
train_wmae = compute_wmae(y_train, pred, is_holiday=X_train['IsHoliday'])
print(f"Train WMAE: {train_wmae:.2f}")

pred = mdl.predict(X_valid)
valid_wmae = compute_wmae(y_valid, pred, is_holiday=X_valid['IsHoliday'])
print(f"Valid WMAE: {valid_wmae:.2f}")

from configs import boosting_models_config, basic_config
import importlib
importlib.reload(boosting_models_config)
importlib.reload(basic_config)

from configs.basic_config import minimal_config

log_to_wandb(
    model = mdl,
    train_score = train_wmae,
    val_score = valid_wmae,
    run_name = "walmart_group_sales_model02",
    artifact_name="group_stat_model",
    artifact_type = "model",
    artifact_description = "Our Model. Implementation can be seen at github",
    config= minimal_config)

Train WMAE: 1166.50
Valid WMAE: 1902.21


train_wmae,▁
val_wmae,▁
train_wmae,1166.49874
val_wmae,1902.21405
